In [20]:
import numpy as np
import pandas as pd

In [21]:
df = pd.read_csv('/content/Constitution Of India.csv')

In [22]:
df.head()

,Articles
0,"1. Name and territory of the Union\n(1) India,..."
1,1. The territories of the States; the Union te...
2,2. Admission or establishment of new States: P...
3,2A. Sikkim to be associated with the Union Rep...
4,3. Formation of new States and alteration of a...


In [23]:
df2 = pd.read_csv('/content/Index.csv', encoding='ISO-8859-1')

In [24]:
df2.head()

,Parts of the Indian Constitution,Subject Mentioned in the Part,Articles in Indian Constitution
0,Part I,Union & Its Territory,Article 1-4
1,Part II,Citizenship,Article 5-11
2,Part III,Fundamental Rights,Article 12-35
3,Part IV,Directive Principles,Article 36-51
4,Part IV A,Fundamental Duties,Article 51A


In [25]:
pip install sentence-transformers

In [26]:
from sentence_transformers import SentenceTransformer, util

In [27]:
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [28]:
df['embeddings'] = df['Articles'].apply(lambda x: model.encode(x, convert_to_tensor=True))

In [29]:
sample1 = "Two siblings inherit an ancestral property after the passing of their parents. While one sibling wishes to sell the property and divide the proceeds, the other wants to retain ownership and preserve the ancestral home. The dispute becomes complicated when the sibling wishing to keep the property refuses to cooperate with any efforts to partition or sell it, citing emotional attachment and family tradition. Legal action becomes necessary to resolve the conflict. The siblings may approach a civil court to either partition the property or settle the dispute under the Hindu Succession Act, 1956, which governs the division of ancestral property among heirs. Additionally, the Indian Partition Act, 1893 may be invoked to determine the division of jointly held property when the co-owners cannot agree on its usage or sale."

In [30]:
sample_embedding = model.encode(sample1, convert_to_tensor=True)

In [31]:
sample_embedding

tensor([-7.2916e-02,  6.0143e-02, -1.3040e-02, -6.9563e-02, -4.7420e-02,
        -3.3266e-02, -4.8335e-02, -1.8229e-02,  1.2757e-03, -2.0087e-02,
         9.6045e-03, -1.6091e-02,  3.9839e-02,  1.8302e-02,  1.1341e-02,
         3.2503e-02, -6.3848e-02,  6.2438e-02, -6.9354e-02,  2.3560e-02,
        -5.3902e-03, -4.4215e-02, -3.3578e-02, -7.5228e-03,  8.1704e-02,
         1.0419e-02,  1.3367e-03, -5.1864e-02,  7.7624e-02, -8.1932e-03,
         9.3962e-02, -3.3511e-02, -1.9296e-02,  1.1271e-01, -7.4675e-04,
        -2.0991e-02, -1.3339e-02,  4.0424e-02,  5.1586e-02, -8.1073e-02,
         3.8947e-02,  1.8779e-02,  3.8934e-02, -5.7304e-02,  4.2394e-02,
         6.7623e-02, -1.6520e-03, -2.0112e-02, -3.9201e-02,  5.2957e-02,
         3.5754e-02, -4.8991e-03,  3.0338e-02,  1.4026e-01,  3.6535e-02,
         4.4624e-02,  2.4010e-02, -4.8380e-02,  6.7944e-04,  7.0490e-02,
         2.3486e-02,  9.0203e-02, -2.1073e-03, -7.0293e-02,  9.1332e-03,
        -4.3310e-02,  8.4577e-02, -1.2144e-02, -3.3

In [32]:
df['similarity'] = df['embeddings'].apply(lambda x: util.cos_sim(sample_embedding, x).item())

In [33]:
df_sorted = df.sort_values(by='similarity', ascending=False)

In [34]:
top_5 = df_sorted.head(5)
for idx, row in top_5.iterrows():
    print(f"Article Index: {idx}, Similarity Score: {row['similarity']}")
    print(f"Article: {row['Articles']}\n")

Article Index: 348, Similarity Score: 0.47995203733444214
Article: 295. Succession to property, assets, rights, liabilities and obligations in other cases
(1) As from the commencement of this Constitution
(a) all property and assets which immediately before such commencement were vested in any Indian State corresponding to a State specified in Part B of the First Schedule shall vest in the Union, if the purposes for which such property and assets were held immediately before such commencement will thereafter be purposes of the Union relating to any of the matters enumerated in the Union List, and
(b) all rights, liabilities and obligations of the Government of any Indian State corresponding to a State specified in Part B of the First Schedule, whether arising out of any contract or otherwise, shall be the rights, liabilities and obligations of the Government of India, if the purposes for which such rights were acquired or liabilities or obligations were incurred before such commencemen

In [35]:
# Select the most similar article
most_similar_row = df_sorted.iloc[0]  # Get the top similar article
most_similar_article = most_similar_row['Articles']

In [36]:
from transformers import pipeline

# Load the generative model, for example, T5 or GPT-2
# For T5, specify the model checkpoint
model_name = "t5-small"  # You can also use "gpt2" for GPT-2
generator = pipeline("text2text-generation", model=model_name)  # Use "text-generation" for GPT-2

# Prepare the input text for the generative model
input_text = f"Situation: {sample1}\n\nMost Similar Article: {most_similar_article}\n\nExplain the law and how it affects the situation in simple terms."

# Generate the explanation
explanation = generator(input_text, max_length=150, num_return_sequences=1)

# Print the generated explanation
print("Simplified Explanation:")
print(explanation[0]['generated_text'])  # For T5
# print(explanation[0])  # For GPT-2, you may just want to access the text directly


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Simplified Explanation:
,,,,,: ( 1 ) Explain the law and how it affects the situation in simple terms. to share the property and divide the proceeds, the other wants to retain ownership and preserve the ancestral home . The dispute becomes complicated when the sibling wishing to keep the property refuses to cooperate with any efforts to partition or sell it . The siblings may approach a civil court to either partition the property or settle the dispute under the Hindu Succession Act, 1956 .


In [37]:
from transformers import pipeline

# Load the generative model (e.g., T5)
model_name = "t5-small"  # or "gpt2" for GPT-2
generator = pipeline("text2text-generation", model=model_name)

# Prepare the input text for the generative model
input_text = (
    f"Given the following situation, explain the relevant law and its implications in simple terms.\n\n"
    f"Situation: {sample1}\n\n"
    f"Most Similar Article: {most_similar_article}\n\n"
    "Please provide a concise explanation of how the law applies to this situation."
)

# Generate the explanation
explanation = generator(
    input_text,
    max_length=200,  # Adjust for desired length
    num_return_sequences=1,
    temperature=0.7,  # Adjust for creativity
    top_k=50  # Limit vocabulary tokens
)

# Print the generated explanation
print("Simplified Explanation:")
print(explanation[0]['generated_text'])


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Token indices sequence length is longer than the specified maximum sequence length for this model (521 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Simplified Explanation:
, the division of ancestral property among heirs.,,,, ( 1 ) Please provide a concise explanation of how the law applies to this situation. the::: to sell the property and divide the proceeds, the other wants to retain ownership and preserve the ancestral home. The dispute becomes complicated when the sibling wishing to keep the property refuses to cooperate with any efforts to partition or sell it, citing emotional attachment and family tradition. Legal action becomes necessary to resolve the


In [40]:
from transformers import pipeline, AutoTokenizer

# Load the generative model (e.g., T5)
model_name = "t5-small"  # or "gpt2" for GPT-2
generator = pipeline("text2text-generation", model=model_name, device=0)  # Use GPU

# Prepare the input text for the generative model
input_text = (
    f"Given the following situation, explain the relevant law and its implications in simple terms.\n\n"
    f"Situation: {sample1}\n\n"
    f"Most Similar Article: {most_similar_article}\n\n"
    "Please provide a concise explanation of how the law applies to this situation and how it affects it."
)

# Load tokenizer to check the input length
tokenizer = AutoTokenizer.from_pretrained(model_name)
input_tokens = tokenizer(input_text, return_tensors="pt")

# Ensure the token length is within the limit
if len(input_tokens['input_ids'][0]) > 512:
    input_text = input_text[:512]  # Truncate or modify to fit within 512 tokens

# Generate the explanation
explanation = generator(
    input_text,
    max_length=200,  # Adjust for desired length
    num_return_sequences=1,
    temperature=0.7,  # Adjust for creativity
    top_k=50,  # Limit vocabulary tokens
    do_sample=True  # Enable sampling
)

# Print the generated explanation
print("Simplified Explanation:")
print(explanation[0]['generated_text'])


Token indices sequence length is longer than the specified maximum sequence length for this model (527 > 512). Running this sequence through the model will result in indexing errors


Simplified Explanation:
: Two siblings inherit an ancestral property after the passing of their parents. The second sibling wishes to keep the property, while the other wishes to retain ownership and preserve the ancestral home. The dispute becomes complicated when the sibling wishing to keep the property refuses to cooperate with any efforts to partition or sell it, citing emotional attachment and family traditiial attachment.


In [42]:
from transformers import pipeline, AutoTokenizer

# Load the generative model (e.g., T5)
model_name = "t5-small"  # or "gpt2" for GPT-2
generator = pipeline("text2text-generation", model=model_name, device=0)  # Use GPU

# Prepare the input text for generating a simplified version of the article
simplified_article_input = (
    f"Given the following article, please provide a simplified version:\n\n"
    f"Article: {most_similar_article}\n\n"
)

# Generate the simplified article
simplified_article_output = generator(
    simplified_article_input,
    max_length=150,  # Adjust for desired length
    num_return_sequences=1,
    temperature=0.7,
    top_k=50,
    do_sample=True
)

# Prepare the input text for generating how the law affects the situation
impact_on_situation_input = (
    f"Given the following situation, explain how the relevant law affects it in simple terms.\n\n"
    f"Situation: {sample1}\n\n"
    f"Most Similar Article: {most_similar_article}\n\n"
    "Please explain how the law applies to this situation."
)

# Generate the explanation for how the law affects the situation
impact_on_situation_output = generator(
    impact_on_situation_input,
    max_length=250,  # Adjust for desired length
    num_return_sequences=1,
    temperature=0.7,
    top_k=50,
    do_sample=True
)

# Print the generated outputs
print("Simplified Article:")
print(simplified_article_output[0]['generated_text'])

print("\nImpact on Situation:")
print(impact_on_situation_output[0]['generated_text'])


Token indices sequence length is longer than the specified maximum sequence length for this model (517 > 512). Running this sequence through the model will result in indexing errors


Simplified Article:
shall be the successor of the Government of the corresponding Indian State (1) As from the commencement of this Constitution (1) From the commencement of this Constitution, the Government of each State specified in Part B of the First Schedule shall, as from the commencement of this Constitution, be the successor of the Government of the corresponding Indian State . the Government of the corresponding Indian State shall, if the purposes for which such property and assets were vested in any Indian State corresponding to a State specified in Part B of the First Schedule shall

Impact on Situation:
, if applicable, in the Indian Partition Act, 1956,, (2) As from the commencement of this Constitution, the Government of each State specified in Part B of the First Schedule shall, as from the commencement of this Constitution, be the successor of the Government of the corresponding Indian State in regards to all property and assets .
